# Getting started

The basic runnable component of Pydra is a *task*. Tasks are conceptually similar to
functions, in that they take inputs, operate on them and then return results. However,
unlike functions, tasks are parameterised before they are executed in a separate step.
This enables parameterised tasks to be linked together into workflows that are checked for
errors before they are executed, and modular execution workers and environments to specified
independently of the task being performed.

Tasks can encapsulate Python functions or shell-commands, or be multi-component workflows,
themselves constructed from task components including nested workflows.

## Preparation

Before we get started, lets set up some test data to play with. Here we create a sample
JSON file in a temporary directory

In [1]:
from pathlib import Path
from tempfile import mkdtemp
from pprint import pprint
import json

JSON_CONTENTS = {'a': True, 'b': 'two', 'c': 3, 'd': [7, 0.55, 6]}

test_dir = Path(mkdtemp())
json_file = test_dir / "test.json"
with open(json_file, "w") as f:
    json.dump(JSON_CONTENTS, f)

Next we create a directory containing 10 randomly generated [NIfTI](https://nifti.nimh.nih.gov/) files

In [2]:
from fileformats.medimage import Nifti1

nifti_dir = test_dir / "nifti"
nifti_dir.mkdir()

for i in range(10):
    Nifti1.sample(nifti_dir, seed=i)  # Create a dummy NIfTI file in the dest. directory

Note that when you run concurrent processes within a Jupyter notebook the following snippet
is also required

In [3]:
import nest_asyncio
nest_asyncio.apply()


## Running your first task

Pre-defined task definitions are installed under the `pydra.tasks.*` namespace by separate
task packages (e.g. `pydra-fsl`, `pydra-ants`, ...). To use a pre-defined task definition

* import the class from the `pydra.tasks.*` package it is in
* instantiate it with appropriate parameters
* "call" resulting object (i.e. `my_task(...)`) to execute it as you would a function 

To demonstrate with an example of loading a JSON file with the
`pydra.tasks.common.LoadJson` task, we first create an example JSON file to test with

Now we can load the JSON contents back from the file using the `LoadJson` task definition
class

In [4]:
# Import the task definition
from pydra.tasks.common import LoadJson

# Instantiate the task definition, providing the JSON file we want to load
load_json = LoadJson(file=json_file)

# Run the task to load the JSON file
outputs = load_json()

# Access the loaded JSON output contents and check they match original
assert outputs.out == JSON_CONTENTS

A newer version (0.25) of nipype/pydra is available. You are using 0.25.dev216+ga56b61e5


## Iterating over inputs

It is straightforward to apply the same operation over a set of inputs using the `split()`
method. For example, if we wanted to re-grid all the NIfTI images stored in a directory,
such as the sample ones generated by the code below

Then we can by importing the `MrGrid` shell-command task from the `pydra-mrtrix3` package
and run it over every NIfTI file in the directory using the `TaskDef.split()` method

In [5]:
from pydra.tasks.mrtrix3.v3_0 import MrGrid

# Instantiate the task definition, "splitting" over all NIfTI files in the test directory
# by splitting the "input" input field over all files in the directory
mrgrid = MrGrid(operation="regrid", voxel=(0.5,0.5,0.5)).split(in_file=nifti_dir.iterdir())

# Run the task to resample all NIfTI files
outputs = mrgrid()

# Print the locations of the output files
pprint(outputs.out_file)

RuntimeError: Error running 'MrGrid' task with ['mrgrid', '/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii', 'regrid', '/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-76ca6bb1847e32a4cc9dc37c609baa0d/out_file.mif', '-voxel', '0.5,0.5,0.5']:

stderr:
mrgrid: [01;31m[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)[0m
mrgrid: [01;31m[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-2 format (sizeof_hdr != 540)[0m
mrgrid: [01;31m[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"[0m
mrgrid: [01;31m[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"[0m


It is also possible to iterate over inputs in pairs/n-tuples. For example, if you wanted to use
different voxel sizes for different images, both the list of images and the voxel sizes
are passed to the `split()` method and their combination is specified by a tuple "splitter"


Note that it is important to use a tuple not a list for the splitter definition in this
case, because a list splitter is interpreted as the split over each combination of inputs
(see [Splitting and combining](../explanation/splitting-combining.html) for more details
on splitters).

In [6]:
mrgrid_varying_vox_sizes = MrGrid(operation="regrid").split(
    ("in_file", "voxel"),
    in_file=nifti_dir.iterdir(),
    # Define a list of voxel sizes to resample the NIfTI files to,
    # the list must be the same length as the list of NIfTI files
    voxel=[
        (1.0, 1.0, 1.0),
        (1.0, 1.0, 1.0),
        (1.0, 1.0, 1.0),
        (0.5, 0.5, 0.5),
        (0.75, 0.75, 0.75),
        (0.5, 0.5, 0.5),
        (0.5, 0.5, 0.5),
        (1.0, 1.0, 1.0),
        (1.25, 1.25, 1.25),
        (1.25, 1.25, 1.25),
    ],
)

outputs = mrgrid_varying_vox_sizes()

pprint(outputs.out_file)

RuntimeError: Error running 'MrGrid' task with ['mrgrid', '/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii', 'regrid', '/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-ab511ef6bff4517710084b47bd36bae0/out_file.mif', '-voxel', '1.0,1.0,1.0']:

stderr:
mrgrid: [01;31m[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)[0m
mrgrid: [01;31m[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-2 format (sizeof_hdr != 540)[0m
mrgrid: [01;31m[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"[0m
mrgrid: [01;31m[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"[0m


## Executing tasks in parallel

By default, Pydra will use the *debug* worker, which executes each task sequentially.
This makes it easier to debug tasks and workflows, however, in most cases, once a workflow
is tested, a concurrent worker is preferable so tasks can be executed in parallel
(see [Workers](./3-advanced-execution.html#Workers)). To use multiple processes on a
workstation, select the `cf` worker option when executing the task/workflow. Additional
keyword arguments, will be passed to the worker initialisation (e.g. `n_procs=4`).

Note that when multiprocessing in Python on Windows and macOS (and good practice on Linux/POSIX
OSs for compatibility), you need to place a `if __name__ == "__main__"` block when
executing in top-level scripts to allow the script to be imported, but not executed,
by subprocesses.

In [7]:
from pydra.tasks.mrtrix3.v3_0 import MrGrid

if __name__ == "__main__":  # <-- Add this block to allow the script to imported by subprocesses
    mrgrid = MrGrid(operation="regrid", voxel=(0.5,0.5,0.5)).split(in_file=nifti_dir.iterdir())
    outputs = mrgrid(worker="cf", n_procs=4)  # <-- Select the "cf" worker here
    print("\n".join(str(p) for p in outputs.out_file))

Task exception was never retrieved
future: <Task finished name='Task-3' coro=<ConcurrentFuturesWorker.run() done, defined at /usr/share/miniconda/lib/python3.12/site-packages/pydra/engine/workers.py:189> exception=RuntimeError('Error running \'MrGrid\' task with [\'mrgrid\', \'/tmp/tmpt9an3fws/nifti/MTX6T5bC6O3hDpHJanr4VjXi.nii\', \'regrid\', \'/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-493775935d04d6631d11949224004ce7/out_file.mif\', \'-voxel\', \'0.5,0.5,0.5\']:\n\nstderr:\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/MTX6T5bC6O3hDpHJanr4VjXi.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)\x1b\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/MTX6T5bC6O3hDpHJanr4VjXi.nii" is not in NIFTI-2 format (sizeof_hdr != 540)\x1b\nmrgrid: \x1b[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/MTX6T5bC6O3hDpHJanr4VjXi.nii"\x1b\nmrgrid: \x1b[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/MTX6T5bC6O3hDpHJanr4VjXi.nii"\x1b\n')>
concurrent.futures.process._R

Task exception was never retrieved
future: <Task finished name='Task-5' coro=<ConcurrentFuturesWorker.run() done, defined at /usr/share/miniconda/lib/python3.12/site-packages/pydra/engine/workers.py:189> exception=RuntimeError('Error running \'MrGrid\' task with [\'mrgrid\', \'/tmp/tmpt9an3fws/nifti/XYEqaPDVxVqXTzHQlIXqXQ0u.nii\', \'regrid\', \'/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-c0e4137c18c07e05b77dac2a94671a0a/out_file.mif\', \'-voxel\', \'0.5,0.5,0.5\']:\n\nstderr:\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/XYEqaPDVxVqXTzHQlIXqXQ0u.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)\x1b\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/XYEqaPDVxVqXTzHQlIXqXQ0u.nii" is not in NIFTI-2 format (sizeof_hdr != 540)\x1b\nmrgrid: \x1b[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/XYEqaPDVxVqXTzHQlIXqXQ0u.nii"\x1b\nmrgrid: \x1b[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/XYEqaPDVxVqXTzHQlIXqXQ0u.nii"\x1b\n')>
concurrent.futures.process._R

Task exception was never retrieved
future: <Task finished name='Task-4' coro=<ConcurrentFuturesWorker.run() done, defined at /usr/share/miniconda/lib/python3.12/site-packages/pydra/engine/workers.py:189> exception=RuntimeError('Error running \'MrGrid\' task with [\'mrgrid\', \'/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii\', \'regrid\', \'/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-76ca6bb1847e32a4cc9dc37c609baa0d/out_file.mif\', \'-voxel\', \'0.5,0.5,0.5\']:\n\nstderr:\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)\x1b\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-2 format (sizeof_hdr != 540)\x1b\nmrgrid: \x1b[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"\x1b\nmrgrid: \x1b[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"\x1b\n')>
concurrent.futures.process._R

Task exception was never retrieved
future: <Task finished name='Task-2' coro=<ConcurrentFuturesWorker.run() done, defined at /usr/share/miniconda/lib/python3.12/site-packages/pydra/engine/workers.py:189> exception=RuntimeError('Error running \'MrGrid\' task with [\'mrgrid\', \'/tmp/tmpt9an3fws/nifti/o7hRfp9mNCCElZfobqz3xhq9.nii\', \'regrid\', \'/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-7afa666d84383040618a0fa98bbf6816/out_file.mif\', \'-voxel\', \'0.5,0.5,0.5\']:\n\nstderr:\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/o7hRfp9mNCCElZfobqz3xhq9.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)\x1b\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/o7hRfp9mNCCElZfobqz3xhq9.nii" is not in NIFTI-2 format (sizeof_hdr != 540)\x1b\nmrgrid: \x1b[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/o7hRfp9mNCCElZfobqz3xhq9.nii"\x1b\nmrgrid: \x1b[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/o7hRfp9mNCCElZfobqz3xhq9.nii"\x1b\n')>
concurrent.futures.process._R

Task exception was never retrieved
future: <Task finished name='Task-7' coro=<ConcurrentFuturesWorker.run() done, defined at /usr/share/miniconda/lib/python3.12/site-packages/pydra/engine/workers.py:189> exception=RuntimeError('Error running \'MrGrid\' task with [\'mrgrid\', \'/tmp/tmpt9an3fws/nifti/ogyjey4XVnHrkgn5ZYXltMT0.nii\', \'regrid\', \'/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-c1b4d4812e300c92de67d72c95a847e8/out_file.mif\', \'-voxel\', \'0.5,0.5,0.5\']:\n\nstderr:\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/ogyjey4XVnHrkgn5ZYXltMT0.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)\x1b\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/ogyjey4XVnHrkgn5ZYXltMT0.nii" is not in NIFTI-2 format (sizeof_hdr != 540)\x1b\nmrgrid: \x1b[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/ogyjey4XVnHrkgn5ZYXltMT0.nii"\x1b\nmrgrid: \x1b[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/ogyjey4XVnHrkgn5ZYXltMT0.nii"\x1b\n')>
concurrent.futures.process._R

...

Task exception was never retrieved
future: <Task finished name='Task-9' coro=<ConcurrentFuturesWorker.run() done, defined at /usr/share/miniconda/lib/python3.12/site-packages/pydra/engine/workers.py:189> exception=RuntimeError('Error running \'MrGrid\' task with [\'mrgrid\', \'/tmp/tmpt9an3fws/nifti/76dfZTPtLLKjAyS96HBqcbCt.nii\', \'regrid\', \'/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-498e9e06b66bfc6e8acb06611ac1bb3c/out_file.mif\', \'-voxel\', \'0.5,0.5,0.5\']:\n\nstderr:\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/76dfZTPtLLKjAyS96HBqcbCt.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)\x1b\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/76dfZTPtLLKjAyS96HBqcbCt.nii" is not in NIFTI-2 format (sizeof_hdr != 540)\x1b\nmrgrid: \x1b[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/76dfZTPtLLKjAyS96HBqcbCt.nii"\x1b\nmrgrid: \x1b[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/76dfZTPtLLKjAyS96HBqcbCt.nii"\x1b\n')>
concurrent.futures.process._R

Task exception was never retrieved
future: <Task finished name='Task-8' coro=<ConcurrentFuturesWorker.run() done, defined at /usr/share/miniconda/lib/python3.12/site-packages/pydra/engine/workers.py:189> exception=RuntimeError('Error running \'MrGrid\' task with [\'mrgrid\', \'/tmp/tmpt9an3fws/nifti/0UAqFzWsDK4FrUMp48Y3tT3Q.nii\', \'regrid\', \'/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-fdc6a8c6810dd12f574f0acd72993235/out_file.mif\', \'-voxel\', \'0.5,0.5,0.5\']:\n\nstderr:\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/0UAqFzWsDK4FrUMp48Y3tT3Q.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)\x1b\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/0UAqFzWsDK4FrUMp48Y3tT3Q.nii" is not in NIFTI-2 format (sizeof_hdr != 540)\x1b\nmrgrid: \x1b[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/0UAqFzWsDK4FrUMp48Y3tT3Q.nii"\x1b\nmrgrid: \x1b[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/0UAqFzWsDK4FrUMp48Y3tT3Q.nii"\x1b\n')>
concurrent.futures.process._R

Task exception was never retrieved
future: <Task finished name='Task-10' coro=<ConcurrentFuturesWorker.run() done, defined at /usr/share/miniconda/lib/python3.12/site-packages/pydra/engine/workers.py:189> exception=RuntimeError('Error running \'MrGrid\' task with [\'mrgrid\', \'/tmp/tmpt9an3fws/nifti/oHwLMeaZqo9DZDNjN1GTPdVK.nii\', \'regrid\', \'/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-9ac88bc2b1899b78ee8d6c90bb226a6e/out_file.mif\', \'-voxel\', \'0.5,0.5,0.5\']:\n\nstderr:\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/oHwLMeaZqo9DZDNjN1GTPdVK.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)\x1b\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/oHwLMeaZqo9DZDNjN1GTPdVK.nii" is not in NIFTI-2 format (sizeof_hdr != 540)\x1b\nmrgrid: \x1b[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/oHwLMeaZqo9DZDNjN1GTPdVK.nii"\x1b\nmrgrid: \x1b[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/oHwLMeaZqo9DZDNjN1GTPdVK.nii"\x1b\n')>
concurrent.futures.process._

Task exception was never retrieved
future: <Task finished name='Task-6' coro=<ConcurrentFuturesWorker.run() done, defined at /usr/share/miniconda/lib/python3.12/site-packages/pydra/engine/workers.py:189> exception=RuntimeError('Error running \'MrGrid\' task with [\'mrgrid\', \'/tmp/tmpt9an3fws/nifti/ujOeHwdFcAefAZhnM6Jy8c1r.nii\', \'regrid\', \'/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-0006560da876a855fb525c2b5595ef91/out_file.mif\', \'-voxel\', \'0.5,0.5,0.5\']:\n\nstderr:\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/ujOeHwdFcAefAZhnM6Jy8c1r.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)\x1b\nmrgrid: \x1b[ERROR] image "/tmp/tmpt9an3fws/nifti/ujOeHwdFcAefAZhnM6Jy8c1r.nii" is not in NIFTI-2 format (sizeof_hdr != 540)\x1b\nmrgrid: \x1b[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/ujOeHwdFcAefAZhnM6Jy8c1r.nii"\x1b\nmrgrid: \x1b[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/ujOeHwdFcAefAZhnM6Jy8c1r.nii"\x1b\n')>
concurrent.futures.process._R

ValueError: Task 'MrGrid' raised an error, full crash report is here: 
/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-76ca6bb1847e32a4cc9dc37c609baa0d/_error.pklz
/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-0006560da876a855fb525c2b5595ef91/_error.pklz
/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-c1b4d4812e300c92de67d72c95a847e8/_error.pklz
/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-9ac88bc2b1899b78ee8d6c90bb226a6e/_error.pklz
/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-493775935d04d6631d11949224004ce7/_error.pklz
/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-fdc6a8c6810dd12f574f0acd72993235/_error.pklz
/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-4fd1fd1c1ff5cdb044e6154ddb4739ec/_error.pklz
/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache/shell-c0e4137c18c07e05b77dac2a94671a0a/_error.pklz
/home/runner/.cache/pydra/0.25.dev216+ga56b61e5/run-cache

## File-system locations

Output and intermediate files are typically generated during the course of a workflow/task run.
In addition to this, Pydra generates a cache directory for each task, in which
the task definition, results and any errors are stored in [cloudpickle](https://github.com/cloudpipe/cloudpickle)
files for future reference (see [Troubleshooting](./troubleshooting.html)).
By default, these cache directories are stored in a platform-specific application-cache
directory

* Windows: `C:\Users\<username>\AppData\Local\pydra\<pydra-version>\run-cache`
* Linux: `/home/<username>/.cache/pydra/<pydra-version>/run-cache`
* macOS: `/Users/<username>/Library/Caches/pydra/<pydra-version>/run-cache`

When a task runs, a unique hash is generated by the combination of all the inputs to the
task and the operation to be performed. This hash is used to name the task cache directory
within the specified cache root. Therefore, if you use the same cache
root and in a subsequent run the same task is executed with the same
inputs, then the path of its cache directory will be the same, and if Pydra finds
existing results at that path, then the outputs generated by the previous run will be
reused.

This cache will grow as more runs are called, therefore care needs to be taken to ensure
there is enough space on the target disk. Since the cache will be constantly To specify
a different location for this cache, simply provide the `cache_root` keyword argument to the execution call

In [8]:
outputs = mrgrid(cache_dir=Path("~/pydra-cache").expanduser())

pprint(outputs)

RuntimeError: Error running 'MrGrid' task with ['mrgrid', '/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii', 'regrid', '/home/runner/pydra-cache/shell-76ca6bb1847e32a4cc9dc37c609baa0d/out_file.mif', '-voxel', '0.5,0.5,0.5']:

stderr:
mrgrid: [01;31m[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)[0m
mrgrid: [01;31m[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-2 format (sizeof_hdr != 540)[0m
mrgrid: [01;31m[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"[0m
mrgrid: [01;31m[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"[0m


To check alternative cache roots, while storing any generated task cache dirs in the 
specified cache root, the `cache_locations` keyword argument can be used

In [9]:
from pydra.utils import default_run_cache_dir

my_cache_dir = Path("~/new-pydra-cache").expanduser()
my_cache_dir.mkdir(exist_ok=True)

outputs = mrgrid(
    cache_dir=my_cache_dir,
    cache_locations=[default_run_cache_dir]
)

print(outputs)

RuntimeError: Error running 'MrGrid' task with ['mrgrid', '/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii', 'regrid', '/home/runner/new-pydra-cache/shell-76ca6bb1847e32a4cc9dc37c609baa0d/out_file.mif', '-voxel', '0.5,0.5,0.5']:

stderr:
mrgrid: [01;31m[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-1.1 format (sizeof_hdr != 348)[0m
mrgrid: [01;31m[ERROR] image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii" is not in NIFTI-2 format (sizeof_hdr != 540)[0m
mrgrid: [01;31m[ERROR] unknown format for image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"[0m
mrgrid: [01;31m[ERROR] error opening image "/tmp/tmpt9an3fws/nifti/i0VpEBOWfbZAVaBSo63bbH6x.nii"[0m
